In [9]:
%matplotlib inline
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import ipywidgets as widgets


In [17]:
rand_ints_output = widgets.Output()
check_output = widgets.Output()

def generate_ints(n, output=None):
    result = np.sort(np.random.uniform(size=n))
    if output is not None:
        output.clear_output()
        with output:
            print(result)
    return result

def pirson(row, alpha, m):
    if m == 0:
        m = int(1 + np.floor(3.22 * np.log10(len(row))))
    min = np.min(row)
    max = np.max(row)
    h = (max - min) / m
    delta_sum = 0
    for i in range(m):
        n = np.sum((row >= min + h * i) & (row <= min + h * (i + 1)))
        p = scipy.stats.uniform.cdf(min + h * (i + 1)) - scipy.stats.uniform.cdf(min + h * i)
        delta = (n - len(row) * p) ** 2 / (len(row) * p)
        delta_sum += delta

    tau = scipy.stats.poisson.ppf(1 - alpha / 2, m - 1)
    print("δ = {:.2f}".format(delta_sum))
    print("tau = {:.2f}".format(tau))
    return np.abs(delta_sum) <= tau


def Dnp(x):
    return np.max([np.abs(i / len(x) - x[i - 1]) for i in range(1, len(x) + 1)])

def Dnm(x):
    return np.max([np.abs(x[i - 1] - (i - 1) / len(x)) for i in range(1, len(x) + 1)])

def Dn(x):
    dnp = Dnp(x)
    dnm = Dnm(x)
    return np.max([dnp, dnm])

def Dn_alpha(n, alpha):
    return np.sqrt(1 / (2 * n) * np.log(2 / (1 - alpha)))

def kolmogorov(row, alpha, m):
    dna = Dn_alpha(m, alpha)
    dn = Dn(row)
    print("D_n = " + str(dn))
    print("D_n (α) = " + str(dna))
    return dn <= dna

In [18]:
num_row = None
N = widgets.IntText(description='N:', value=10)
alpha = widgets.BoundedFloatText(description='α:', value=0.05, min=0.001, max=1)
def generate_and_check_btn_handler(btn):
    num_row = generate_ints(N.value, rand_ints_output)
    check_output.clear_output()
    with check_output:
        p = pirson(num_row, alpha.value, N.value)
        k = kolmogorov(num_row, alpha.value, N.value)
        print("Пирсон: " + ("H0" if p else "H1"))
        print("Колмогоров: " + ("H0" if k else "H1"))

def mk_btn(description, action):
    btn = widgets.Button(description=description, layout=widgets.Layout(width="200px"))
    btn.on_click(action)
    return btn

check_button = mk_btn("Сгенерировать и проверить", generate_and_check_btn_handler)

In [19]:
widgets.VBox((N, alpha, check_button))

In [20]:
check_output

Output()

In [21]:
rand_ints_output

Output()